In [1]:
using Apophis, BenchmarkTools
using Symbolics: build_function, jacobian, derivative, Num, @variables
using LinearAlgebra
using SparseArrays: sparse, SparseMatrixCSC
using DifferentialEquations: ODEProblem, solve
using Sundials: CVODE_BDF

In [4]:
g = [2 3 6
    3 9 8
     6 8 25]

3×3 Matrix{Int64}:
 2  3   6
 3  9   8
 6  8  25

In [5]:
P₀ = only(@variables P₀)
T = only(@variables T)
Y = collect(only(@variables Y[1:53]));
T₀ = only(@variables T₀)
Y₀ = collect(only(@variables Y₀[1:53]));

In [21]:
init(:H2, 1000.0, 1.59e6, H2 = 0.29, N2 = 0.56, O2 = 0.15)
#init(:GRI3, 1000.0, CH4 = 0.05, N2 = 0.75, O2 = 0.20)
#init(:H2, T₀, P₀, Y₀);
#init(:GRI3, T₀, P₀, Y₀)

In [22]:
## Symbolically
#step!(gas, Y₀, T₀)
#step!(gas, Y, T)

## Numerically
step!(gas, gas.initial.mass_fractions, gas.initial.temperature)
#step!(gas, gas.current.mass_fractions, gas.current.temperature)

In [23]:
sol, _ = equilibrate(4, gas)

(SciMLBase.ODESolution{Float64, 2, Vector{Vector{Float64}}, Nothing, Nothing, Vector{Float64}, Nothing, ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, Tuple{Apophis.Gas{Float64}, Vector{Float64}, Vector{Float64}}, SciMLBase.ODEFunction{true, typeof(Apophis.IdealGasReactor!), UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}, CVODE_BDF{:Newton, :Dense, Nothing, Nothing}, SciMLBase.HermiteInterpolation{Vector{Float64}, Vector{Vector{Float64}}, Vector{Vector{Float64}}}, DiffEqBase.DEStats}([[0.0, 0.29, 0.0, 0.15, 0.0, 0.0, 0.56, 0.0, 0.0, 0.0, 1000.0], [2.3766404482631687e-14, 0.2899999999999524, 1.1064762262023677e-16, 0.14999999999924432, 1.0201813884758172e-16, 2.6997650417823518e-17, 0.56, 7.792309969217862e-13, 1.1774618711879953e-17, 0.0, 999.9999999985475

In [25]:
using CairoMakie


┌ Warning: unknown command \Join
└ @ MathTeXEngine ~/.julia/packages/MathTeXEngine/ZP0gS/src/parser/commands_registration.jl:48


In [30]:
lines(sol.t, sol[end, :])

### $$ A = \begin{bmatrix} \bm{a} & \bm{b}\\ \bm{c} &d \end{bmatrix} $$ 

$$ \bm{a} = \frac{∂\bm{Ẏ}}{∂\bm{Y}}\ \ [ns\times ns], \ \ \bm{b} = \frac{d\bm{Ẏ}}{dT}\ \ [ns\times 1]$$
$$ \bm{c} = \frac{d\bm{Ẏ}}{dT} \ \ [1\times ns], \ \ d =  \frac{dṪ}{dT}\ \ [1\times 1]$$

In [26]:
dṪ = only(gas.intermediate.temperature_change_rate)
dẎ = gas.intermediate.mass_change_rate
q = vcat(dẎ, dṪ)
u = vcat(Y, T)

A = jacobian(q, u);
_, A_expr = build_function(A, Y, T, Y₀, T₀, P₀)
A_function = eval(A_expr);

### <center>Jacobian Trials</center>

In [17]:
A = zeros(54, 54);
init(:GRI3, 1000.0, CH4 = 0.05, N2 = 0.75, O2 = 0.20)
step!(gas, gas.initial.mass_fractions, gas.initial.temperature)

In [50]:
tstart = time()
sol = backward(gas)
tend = time()
println(tend - tstart, " - ", sol.retcode)

  37.374 ms (24101 allocations: 1.61 MiB)


retcode: Success
Interpolation: 3rd order Hermite
t: 35-element Vector{Float64}:
 0.9
 0.8989302861365588
 0.7977613678546672
 0.44085621706059536
 0.40677059535453586
 0.37268497364847636
 0.33859935194241686
 0.30451373023635736
 0.2971285478802605
 0.28974336552416363
 0.28235818316806677
 0.26932993512087167
 0.25630168707367657
 ⋮
 0.1656558492785666
 0.16194672469111526
 0.15823760010366392
 0.15149018416701754
 0.14474276823037116
 0.13799535229372478
 0.1312479363570784
 0.1241820967643194
 0.1171162571715604
 0.11005041757880141
 0.10298457798604241
 0.1
u: 35-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [-9.81232078010744e-13, 7.143766504941305e-18, -5.20933826662977e-13, -8.346905665476383e-14, -4.742699218925251e-13, 5.350598900883399e-19, -9.059142557082964e-23, 1.4310387831890401e-13, 2.0133399871538316e-13, 1.3999887690201916e-23, -4.8640637436282345e-16]
 [-9.687660252043518e-10, 7.197252943303082e-15, -1.4120491107931102e-1

In [8]:
open("./jac/H2.jl", "a") do io
   println(io, A_expr)
   #replace(io, "(ˍ₋out, ˍ₋arg1, T, ˍ₋arg3, T₀, P₀)" => "jac(ˍ₋out, ˍ₋arg1, T, ˍ₋arg3, T₀, P₀)")
end